# 📊 Fase 2 – Dataset y Preprocesamiento

## 🎯 Objetivo
Preparar un conjunto de datos limpio y estructurado para entrenar un modelo de Machine Learning enfocado en la detección de intrusiones en redes, como parte del proyecto modular **CiberVigIA 2025B**.

## 📁 Datasets sugeridos
- **NSL-KDD**: Dataset clásico para sistemas de detección de intrusos (IDS), útil para benchmarking.
- **CICIDS 2017**: Dataset más realista y contemporáneo, con tráfico simulado que incluye ataques modernos.

## 🧪 Pasos técnicos

1. **Descarga del dataset**  
   - Formato: `.csv`  
   - Fuente: Repositorio oficial o mirror institucional

2. **Limpieza de datos**  
   - Eliminación de columnas irrelevantes (por ejemplo: `timestamp`, `flow_id`, etc.)  
   - Revisión de valores nulos o inconsistentes  
   - Normalización de nombres de columnas

3. **Codificación de variables categóricas**  
   - Ejemplo: `protocol_type` → TCP / UDP / ICMP  
   - Técnicas: One-Hot Encoding o Label Encoding según el modelo a utilizar

4. **Verificación de balance de clases**  
   - Identificación de clases mayoritarias/minoritarias  
   - Posible aplicación de técnicas como SMOTE para balancear

5. **Exportación del dataset preprocesado**  
   - Formato final: `CSV` limpio y listo para entrenamiento  
   - Ubicación: `data/processed/` dentro del repositorio

## 📦 Entregable
Un archivo `.csv` con los datos preprocesados, documentado y listo para ser utilizado en la Fase 3 (Entrenamiento del modelo ML).

---

> 🧠 *Este notebook forma parte del pipeline modular del proyecto CiberVigIA, alineado con objetivos de sostenibilidad, seguridad informática y análisis de tráfico de red mediante inteligencia artificial.*


In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE, RandomOverSampler
import numpy as np

Función de ayuda para obtener los nombres de las columnas en el dataset de KDD.

In [2]:
def get_column_names(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    column_names = []
    attack_types_raw = lines[0].strip()  # Línea 0: tipos de ataques
    attack_types = [attack.strip() for attack in attack_types_raw.split(',')]  # Parsear en lista
    for line in lines:  # Empieza desde línea 2
        if ':' in line:  # Ignora líneas sin ":"
            name = line.split(':')[0].strip()  # Toma antes de ":", quita espacios
            column_names.append(name)
    column_names += ['label']
    return attack_types, column_names

### Paso 1. Carga de Datasets

In [3]:
df = pd.read_csv('../../data/raw/KDD CUP 99/corrected.gz', header=None)
column_names = get_column_names('../../data/raw/KDD CUP 99/kddcup.names')
attack_types, df.columns = column_names

In [4]:
data1 = pd.read_csv('../../data/raw/IDS 2017/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')
data2 = pd.read_csv('../../data/raw/IDS 2017/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv')
data3 = pd.read_csv('../../data/raw/IDS 2017/Friday-WorkingHours-Morning.pcap_ISCX.csv')
data4 = pd.read_csv('../../data/raw/IDS 2017/Monday-WorkingHours.pcap_ISCX.csv')
data5 = pd.read_csv('../../data/raw/IDS 2017/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv')
data6 = pd.read_csv('../../data/raw/IDS 2017/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv')
data7 = pd.read_csv('../../data/raw/IDS 2017/Tuesday-WorkingHours.pcap_ISCX.csv')
data8 = pd.read_csv('../../data/raw/IDS 2017/Wednesday-workingHours.pcap_ISCX.csv')

### Paso 2. Agrupar Dataset de CIC

In [5]:
data_list = [data1, data2, data3, data4, data5, data6, data7, data8]

In [6]:
print('Data dimensions: ')
for i, data in enumerate(data_list, start = 1):
  rows, cols = data.shape
  print(f'Data{i} -> {rows} rows, {cols} columns')

Data dimensions: 
Data1 -> 225745 rows, 79 columns
Data2 -> 286467 rows, 79 columns
Data3 -> 191033 rows, 79 columns
Data4 -> 529918 rows, 79 columns
Data5 -> 288602 rows, 79 columns
Data6 -> 170366 rows, 79 columns
Data7 -> 445909 rows, 79 columns
Data8 -> 692703 rows, 79 columns


In [7]:
data = pd.concat(data_list) # Dataset CIC 2017 concatenado
rows, cols = data.shape

In [8]:
print('New dimension for CIC Dataset:')
print(f'Number of rows: {rows}')
print(f'Number of columns: {cols}')
print(f'Total cells: {rows * cols}')

New dimension for CIC Dataset:
Number of rows: 2830743
Number of columns: 79
Total cells: 223628697


In [9]:
for d in data_list: del d # Limpieza

In [10]:
rows, cols = df.shape
print(f'Data KDD -> {rows} rows, {cols} columns')

Data KDD -> 311029 rows, 42 columns


### Paso 3: Mostrar los nombres de las columnas y exploración

In [11]:
print(f'Data KDD -> {df.columns}')
print(f'Data CIC -> {data.columns}')

Data KDD -> Index(['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
       'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
       'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
       'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
       'num_access_files', 'num_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate', 'label'],
      dtype='object')
Data CIC -> Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Pa

In [12]:
data.columns = data.columns.str.lstrip()

Limpiamos las columnas no necesarias de cada uno:

In [14]:
non_capturable = [
    'hot', # Host-Based
    'num_failed_logins', # Requiere logs host.
    'logged_in', # Host-Based
    'num_compromised', # Correlación externa necesatia 
    'root_shell', # No es fiable
    'su_attempt', # Depende de payload
    'num_root', # Requiere logs host.
    'num_file_creations',  # Requiere host monitoring o inspección profunda.
    'num_shells', # No fiable en encriptado
    'num_access_files', # Host-Based
    'num_outbound_cmds', # Parsing profundo
    'is_host_login', # No siempre capturable
    'is_guest_login', # No siempre capturable
]

df = df.drop(columns=[col for col in non_capturable if col in df.columns])
print("No capturables quitadas. Nueva forma:", df.shape)

No capturables quitadas. Nueva forma: (311029, 30)


In [15]:
print('Data KDD:')
print("Forma original:", df.shape)
print("Tipos:", df.dtypes)
print("Missing values:", df.isnull().sum().sum())
print("Duplicados:", df.duplicated().sum())
label_counts = df['label'].value_counts() if 'label' in df else pd.Series()
print("Balance de labels:", label_counts)

Data KDD:
Forma original: (311029, 30)
Tipos: duration                         int64
protocol_type                   object
service                         object
flag                            object
src_bytes                        int64
dst_bytes                        int64
land                             int64
wrong_fragment                   int64
urgent                           int64
su_attempted                     int64
count                            int64
srv_count                        int64
serror_rate                    float64
srv_serror_rate                float64
rerror_rate                    float64
srv_rerror_rate                float64
same_srv_rate                  float64
diff_srv_rate                  float64
srv_diff_host_rate             float64
dst_host_count                   int64
dst_host_srv_count               int64
dst_host_same_srv_rate         float64
dst_host_diff_srv_rate         float64
dst_host_same_src_port_rate    float64
dst_host_srv_diff_

Filtrar clases con pocas muestras (evita error SMOTE)

In [16]:
low_sample_classes = label_counts[label_counts < 2].index
if len(low_sample_classes) > 0:
    print(f"Filtrando clases con <2 muestras: {low_sample_classes}")
    df = df[~df['label'].isin(low_sample_classes)]

Filtrando clases con <2 muestras: Index(['imap.'], dtype='object', name='label')


In [15]:
print('Data CIC:')
print("Forma original:", data.shape)
print("Tipos:", data.dtypes)
print("Missing values:", data.isnull().sum().sum())
print("Duplicados:", data.duplicated().sum())
label_counts = data['Label'].value_counts() if 'Label' in data else pd.Series()
print("Balance de labels:", label_counts)

Data CIC:
Forma original: (2830743, 79)
Tipos: Destination Port                 int64
Flow Duration                    int64
Total Fwd Packets                int64
Total Backward Packets           int64
Total Length of Fwd Packets      int64
                                ...   
Idle Mean                      float64
Idle Std                       float64
Idle Max                         int64
Idle Min                         int64
Label                           object
Length: 79, dtype: object
Missing values: 1358
Duplicados: 308381
Balance de labels: Label
BENIGN                        2273097
DoS Hulk                       231073
PortScan                       158930
DDoS                           128027
DoS GoldenEye                   10293
FTP-Patator                      7938
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1966
Web Attack � Brute Force         1507
Web Attack � XS

Filtrar clases con pocas muestras (evita error SMOTE)

In [16]:
low_sample_classes = label_counts[label_counts < 2].index
if len(low_sample_classes) > 0:
    print(f"Filtrando clases con <2 muestras: {low_sample_classes}")
    data = data[~data['Label'].isin(low_sample_classes)]

### Paso 4: Limpiamos los datos perdidos / duplicados / infinitos

In [17]:
df = df.replace([np.inf, -np.inf], np.nan).fillna(0).drop_duplicates()
data = data.replace([np.inf, -np.inf], np.nan).fillna(0).drop_duplicates()

### Paso 5: Agegamos encodings categoricos

In [18]:
lekdd = LabelEncoder()
for col in ['protocol_type', 'service', 'flag']:
    df[col] = lekdd.fit_transform(df[col])

In [19]:
lecic = LabelEncoder()
for col in data.select_dtypes(include=['object']).columns:
    if col != 'Label':
        data[col] = lecic.fit_transform(data[col])

### Paso 6: Scaling numerico

In [19]:
numerical_cols_kdd = df.select_dtypes(include=['int64', 'float64']).columns.drop('label', errors='ignore')
scaler_kdd = StandardScaler()
df[numerical_cols_kdd] = scaler_kdd.fit_transform(df[numerical_cols_kdd])

In [21]:
numerical_cols_cic = data.select_dtypes(include=['int64', 'float64']).columns.drop('Label', errors='ignore')
scaler_cic = StandardScaler()
data[numerical_cols_cic] = scaler_cic.fit_transform(data[numerical_cols_cic])

### Paso 7: Balanceo (SMOTE) -> Se necesita en ambos datasets

In [20]:
def smote_por_chunks(X, y, chunk_size=50000, random_state=42, k_neighbors=1):
    """
    Aplica SMOTE por bloques. Si SMOTE falla en un chunk, usa RandomOverSampler como respaldo.
    Retorna un DataFrame balanceado.
    """
    X_res_total, y_res_total = [], []

    # Identifica la clase minoritaria
    clase_min = y.value_counts().idxmin()

    # Separa las clases
    X_min, y_min = X[y == clase_min], y[y == clase_min]
    X_maj, y_maj = X[y != clase_min], y[y != clase_min]

    # Itera sobre la clase mayoritaria en bloques
    for i in range(0, len(X_maj), chunk_size):
        X_chunk = pd.concat([X_min, X_maj.iloc[i:i + chunk_size]])
        y_chunk = pd.concat([y_min, y_maj.iloc[i:i + chunk_size]])

        try:
            # Intenta aplicar SMOTE
            smote = SMOTE(random_state=random_state, k_neighbors=k_neighbors)
            X_res, y_res = smote.fit_resample(X_chunk, y_chunk)
            print(f"Chunk {i}: SMOTE aplicado correctamente.")
        except Exception as e:
            # Si SMOTE falla, usa RandomOverSampler
            print(f"Chunk {i} falló con SMOTE: {e}. Usando RandomOverSampler.")
            ros = RandomOverSampler(random_state=random_state)
            X_res, y_res = ros.fit_resample(X_chunk, y_chunk)

        # Guarda los resultados del chunk
        X_res_total.append(pd.DataFrame(X_res, columns=X.columns))
        y_res_total.append(pd.Series(y_res))

    # Une todos los chunks procesados
    X_final = pd.concat(X_res_total, ignore_index=True)
    y_final = pd.concat(y_res_total, ignore_index=True)

    return pd.concat([X_final, pd.Series(y_final, name='Label')], axis=1)

In [21]:
X = df.drop('label', axis=1)
y = df['label']
df_balanced = smote_por_chunks(X, y)

Chunk 0 falló con SMOTE: Expected n_neighbors <= n_samples_fit, but n_neighbors = 2, n_samples_fit = 1, n_samples = 1. Usando RandomOverSampler.
Chunk 50000 falló con SMOTE: Expected n_neighbors <= n_samples_fit, but n_neighbors = 2, n_samples_fit = 1, n_samples = 1. Usando RandomOverSampler.


In [28]:
X = data.drop('Label', axis=1)
y = data['Label']
data_balanced = smote_por_chunks(X, y)

Chunk 0: SMOTE aplicado correctamente.
Chunk 50000: SMOTE aplicado correctamente.
Chunk 100000: SMOTE aplicado correctamente.
Chunk 150000: SMOTE aplicado correctamente.
Chunk 200000: SMOTE aplicado correctamente.
Chunk 250000: SMOTE aplicado correctamente.
Chunk 300000: SMOTE aplicado correctamente.
Chunk 350000: SMOTE aplicado correctamente.
Chunk 400000: SMOTE aplicado correctamente.
Chunk 450000: SMOTE aplicado correctamente.
Chunk 500000: SMOTE aplicado correctamente.
Chunk 550000: SMOTE aplicado correctamente.
Chunk 600000: SMOTE aplicado correctamente.
Chunk 650000: SMOTE aplicado correctamente.
Chunk 700000: SMOTE aplicado correctamente.
Chunk 750000: SMOTE aplicado correctamente.
Chunk 800000: SMOTE aplicado correctamente.
Chunk 850000: SMOTE aplicado correctamente.
Chunk 900000: SMOTE aplicado correctamente.
Chunk 950000: SMOTE aplicado correctamente.
Chunk 1000000: SMOTE aplicado correctamente.
Chunk 1050000: SMOTE aplicado correctamente.
Chunk 1100000: SMOTE aplicado correc

### Paso 8: Guardar

In [22]:
def guardar_por_chunks(df, ruta_csv, chunk_size=100000):
    # Si el archivo ya existe, lo eliminamos para empezar limpio
    import os
    if os.path.exists(ruta_csv):
        os.remove(ruta_csv)

    # Guardar por bloques
    for i in range(0, len(df), chunk_size):
        chunk = df.iloc[i:i+chunk_size]
        # Solo escribe el encabezado en el primer chunk
        chunk.to_csv(ruta_csv, mode='a', index=False, header=(i==0))
        print(f"Chunk {i} guardado: filas {i} a {i+len(chunk)-1}")


In [23]:
guardar_por_chunks(df_balanced, '../../data/processed/nsl_kdd_clean.csv')
print("NSL-KDD limpiado guardado por chunks.")

Chunk 0 guardado: filas 0 a 99999
Chunk 100000 guardado: filas 100000 a 199999
Chunk 200000 guardado: filas 200000 a 299999
Chunk 300000 guardado: filas 300000 a 399999
Chunk 400000 guardado: filas 400000 a 499999
Chunk 500000 guardado: filas 500000 a 599999
Chunk 600000 guardado: filas 600000 a 699999
Chunk 700000 guardado: filas 700000 a 799999
Chunk 800000 guardado: filas 800000 a 899999
Chunk 900000 guardado: filas 900000 a 999999
Chunk 1000000 guardado: filas 1000000 a 1099999
Chunk 1100000 guardado: filas 1100000 a 1199999
Chunk 1200000 guardado: filas 1200000 a 1299999
Chunk 1300000 guardado: filas 1300000 a 1399999
Chunk 1400000 guardado: filas 1400000 a 1450115
NSL-KDD limpiado guardado por chunks.


In [36]:
guardar_por_chunks(data_balanced, '../../data/processed/cic_ids2017_clean.csv')
print("CIC-IDS2017 limpiado guardado por chunks.")

Chunk 0 guardado: filas 0 a 99999
Chunk 100000 guardado: filas 100000 a 199999
Chunk 200000 guardado: filas 200000 a 299999
Chunk 300000 guardado: filas 300000 a 399999
Chunk 400000 guardado: filas 400000 a 499999
Chunk 500000 guardado: filas 500000 a 599999
Chunk 600000 guardado: filas 600000 a 699999
Chunk 700000 guardado: filas 700000 a 799999
Chunk 800000 guardado: filas 800000 a 899999
Chunk 900000 guardado: filas 900000 a 999999
Chunk 1000000 guardado: filas 1000000 a 1099999
Chunk 1100000 guardado: filas 1100000 a 1199999
Chunk 1200000 guardado: filas 1200000 a 1299999
Chunk 1300000 guardado: filas 1300000 a 1399999
Chunk 1400000 guardado: filas 1400000 a 1499999
Chunk 1500000 guardado: filas 1500000 a 1599999
Chunk 1600000 guardado: filas 1600000 a 1699999
Chunk 1700000 guardado: filas 1700000 a 1799999
Chunk 1800000 guardado: filas 1800000 a 1899999
Chunk 1900000 guardado: filas 1900000 a 1999999
Chunk 2000000 guardado: filas 2000000 a 2099999
Chunk 2100000 guardado: filas 210